In [ ]:
# Install gdown if not already installed
!pip install gdown

# Step 1: Download the preprocessed data
!gdown --id 1JQWAMjNb-iCm8nQUZeo5B2VrmLcjEk6Q -O preprocessed.zip

# Step 2: Unzip the preprocessed data
import shutil
shutil.unpack_archive('preprocessed.zip')

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1JQWAMjNb-iCm8nQUZeo5B2VrmLcjEk6Q
From (redirected): https://drive.google.com/uc?id=1JQWAMjNb-iCm8nQUZeo5B2VrmLcjEk6Q&confirm=t&uuid=3999c255-43cb-48e7-be47-cd5c049a242b
To: /content/preprocessed.zip
100% 63.2M/63.2M [00:02<00:00, 31.2MB/s]


In [ ]:
import pandas as pd

train_df = pd.read_csv('train_processed.csv', na_filter = False)
test_df = pd.read_csv('test_processed.csv', na_filter = False)

In [ ]:
combined_df = pd.concat([train_df, test_df], ignore_index=True)
print("Combined dataset shape:", combined_df.shape)

Combined dataset shape: (223549, 12)


In [ ]:
combined_df['is_toxic'] = combined_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0


In [ ]:
combined_df.to_parquet('combined_df.parquet', index=False)


In [ ]:
from gensim import corpora, models
import joblib


tokenized_texts = combined_df['cleaned_comment_text'].apply(lambda x: x.split())

dictionary = corpora.Dictionary(tokenized_texts)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

corpus_bow = [dictionary.doc2bow(text) for text in tokenized_texts]



dictionary.save('/gensim_dictionary.dict')
joblib.dump(corpus_bow, 'gensim_corpus.joblib')

print("Gensim dictionary and corpus saved!")


Gensim dictionary and corpus saved!


In [ ]:
from gensim import corpora, models
import joblib

toxic_comments = combined_df[combined_df['is_toxic']]

tokenized_toxic = toxic_comments['cleaned_comment_text'].apply(lambda x: x.split())

toxic_dictionary = corpora.Dictionary(tokenized_toxic)
toxic_dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

toxic_corpus_bow = [toxic_dictionary.doc2bow(text) for text in tokenized_toxic]


toxic_dictionary.save('toxic_gensim_dictionary.dict')
joblib.dump(toxic_corpus_bow, 'toxic_gensim_corpus_bow.joblib')

print("Gensim corpus for toxic comments saved!")


TF-IDF model and Gensim corpus for toxic comments saved!
